In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

torch.manual_seed(22)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

data_dir = '/youdir'
model_dir = '/youdir'

fnames = ['dota_train_23.pt', 'dota_val_23.pt', 'dota_test_23.pt']

In [ ]:
from utils import load_data
BATCH_SIZE = 512
_, _, test_dloader = load_data(data_dir, fnames, batch_size=BATCH_SIZE)

In [ ]:
model = torch.load(os.path.join(model_dir, '/yourdir.pt'))

In [ ]:
def eval(model, dloader, device):

    model.eval()
    pickban_hits = 0
    pickban_hits5 = 0
    teamwin_hits = 0
    total_pickban_elements = 0
    total_teamwin_elements = 0

    with torch.no_grad():
        for batch in dloader:
            inputs = batch
            inputs = inputs.to(device)
            targets = inputs[:, 1:]
            outputs = model(inputs)
            scores = torch.argmax(outputs[:, :-1, :], dim=-1)
            _, scores5_idcs = torch.topk(outputs[:, :-1, :], 5, dim=-1)

            # Calculate picks and bans predictions
            pickban_scores, targets_pickbans = scores[:, :-1], targets[:, :-1]
            pickban_hits += (pickban_scores == targets_pickbans).sum().item()
            total_pickban_elements += targets_pickbans.numel()

            # Calculate acc@5 predictions
            pickban_hits5 += (targets.unsqueeze(-1) == scores5_idcs).any(dim=-1).sum().item()

            # Calculates winrate scores
            teamwin_scores, targets_teamwin = scores[:, -1], targets[:, -1]
            teamwin_hits += (teamwin_scores == targets_teamwin).sum().item()
            total_teamwin_elements += targets_teamwin.numel()

    pickban_acc = pickban_hits/total_pickban_elements
    pickban5_acc = pickban_hits5/total_pickban_elements
    win_acc = teamwin_hits/total_teamwin_elements

    print(f'Accuracy on picks and bans: {pickban_acc}')
    print(f'Accuracy @ 5 on picks and bans: {pickban5_acc}')
    print(f'Accuracy on winning team: {win_acc}')

    return pickban_acc, win_acc

In [ ]:
a,b = eval(model, test_dloader, device)